In [ ]:
import numpy as numpy
import tensorflow as tf
from tensorflow.keras import layers, Model

In [ ]:
NUM_DAY = 7
NUM_ROUTE = 3
NUM_STOPS = 2
NUM_FEATURES = NUM_DAY + NUM_ROUTE + NUM_STOPS + 1 + 1

In [ ]:
def apply_mask(x, mask):
    return x * mask

def build_encoder(input_dim, latent_dim=8):
    inputs = layers.Input(shape=(input_dim,))
    x = layers.Dense(64, activation='relu')(inputs)
    x = layers.Dense(32, activation='relu')(x)
    z = layers.Dense(latent_dim)(x)
    return Model(inputs, z, name='encoder')

def build_decoder(latent_dim, output_dim):
    z = layers.Input(shape=(latent_dim,))
    x = layers.Dense(32, activation='relu')(z)
    x = layers.Dense(64, activation='relu')(x)

    features_out = layers.Dense(output_dim, name='features_out')(x)
    travel_time_out = layers.Dense(1, name='travel_time_out')(x)
    pleasure_out = layers.Dense(1, name='pleasure_out')(x)

    return Model(z, [features_out, travel_time_out, pleasure_out], name='decoder')

latent_dim = 8

feature_input = layers.Input(shape=(NUM_FEATURES,), name='features')
mask_input = layers.Input(shape=(NUM_FEATURES,), name='mask')

masked_features = layers.Multiply()([feature_input, mask_input])
model_input = layers.Concatenate()([masked_features, mask_input])

encoder = build_encoder(model_input.shape[-1], latent_dim)
decoder = build_decoder(latent_dim, NUM_FEATURES)

z = encoder(model_input)
features_hat, travel_time_hat, pleasure_hat = decoder(z)

model = Model(
    inputs=[feature_input, mask_input],
    outputs=[features_hat, travel_time_hat, pleasure_hat],
    name='autoencoder')

def combined_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss={
        "features_out": "mse",
        "travel_time_out": "mse",
        "pleasure_out": "mse"
    },
    loss_weights={
        "features_out": 1.0,
        "travel_time_out": 1.0,
        "pleasure_out": 0.5
    }
    )

In [ ]:
# train
import numpy as np

X = np.random.rand(500, NUM_FEATURES)
mask = np.ones_like(X)

travel_time = np.random.rand(500, 1)
pleasure = np.random.rand(500, 1)

model.fit(
    [X, mask],
    {
        "features_out": X,
        "travel_time_out": travel_time,
        "pleasure_out": pleasure
    },
    epochs=30,
    batch_size=32
)

In [ ]:
# inference

x_query = np.zeros((1, NUM_FEATURES))
mask_query = np.zeros((1, NUM_FEATURES))

x_query[0, 0] = 1 # Monday
mask_query[0, 0] = 1

END_TIME_INDEX = NUM_DAY + 1
x_query[0, END_TIME_INDEX] = 540 # 9AM
mask_query[0, END_TIME_INDEX] = 1

def generate_optimal(x, mask, time_weight=1.0, pleasure_weight=1.0, n_samples=50):
    candidates = []

    for _ in range(n_samples):
        f_hat, t_hat, p_hat = model.predict([x, mask], verbose=0)

        score = (
            -time_weight * t_hat[0, 0] +
            pleasure_weight * p_hat[0, 0]
        )

        candidates.append((score, f_hat, t_hat, p_hat))
    
    best = max(candidates, key=lambda c: c[0])
    return best

best_score, best_features, best_time, best_pleasure = generate_optimal(x_query, mask_query)